# Housekeeping

In [0]:
!pip install pillow

In [1]:
from time import gmtime, strftime
from google.colab import drive

print("sync at " + strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))
drive.mount('/content/drive')
image_size = 208    # image size that you will use for your network (input images will be resampled to this size), lower if you have troubles on your laptop (hint: use io.imshow to inspect the quality of the resampled images before feeding it into your network!)

sync at Sat, 22 Jun 2019 18:13:33 +0000


**If regenerating dataset: download dataset**

In [0]:
!wget -nc http://host.robots.ox.ac.uk/pascal/VOC/voc2009/VOCtrainval_11-May-2009.tar
!tar -xf "/content/VOCtrainval_11-May-2009.tar"

--2019-06-15 15:40:13--  http://host.robots.ox.ac.uk/pascal/VOC/voc2009/VOCtrainval_11-May-2009.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 935534080 (892M) [application/x-tar]
Saving to: ‘VOCtrainval_11-May-2009.tar’

VOCtrainval_11-May- 100%[===================>] 892.19M  13.3MB/s    in 66s     

2019-06-15 15:41:20 (13.4 MB/s) - ‘VOCtrainval_11-May-2009.tar’ saved [935534080/935534080]



Getting pregenerated dataset from drive

In [5]:
from time import gmtime, strftime
from google.colab import drive

print("sync at " + strftime("%a, %d %b %Y %H:%M:%S +0000", gmtime()))
drive.mount('/content/drive')
image_size = 208    # image size that you will use for your network (input images will be resampled to this size), lower if you have troubles on your laptop (hint: use io.imshow to inspect the quality of the resampled images before feeding it into your network!)

from lxml import etree
import numpy as np
import os
from skimage import io
from skimage.transform import resize
from skimage.filters import threshold_otsu
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
input_img = Input(shape=(image_size, image_size, 3)) 
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard


x_train = np.load("/content/drive/My Drive/cvproj/dataset/x_train.npy")
y_train = np.load("/content/drive/My Drive/cvproj/dataset/y_train.npy")
x_strain = np.load("/content/drive/My Drive/cvproj/dataset/x_strain.npy")
y_strain = np.load("/content/drive/My Drive/cvproj/dataset/y_strain.npy")
x_val = np.load("/content/drive/My Drive/cvproj/dataset/x_val.npy")
y_val = np.load("/content/drive/My Drive/cvproj/dataset/y_val.npy")
x_sval = np.load("/content/drive/My Drive/cvproj/dataset/x_sval.npy")
y_sval = np.load("/content/drive/My Drive/cvproj/dataset/y_sval.npy")

sync at Sat, 22 Jun 2019 20:49:04 +0000
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The tensorboard module is not an IPython extension.


UsageError: Line magic function `%tensorboard` not found.


In [61]:
%tensorboard

UsageError: Line magic function `%tensorboard` not found.


Get tensorboard support

Regenerating dataset: only if downloaded fresh

In [0]:
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
input_img = Input(shape=(image_size, image_size, 3)) 
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

In [0]:
from lxml import etree
import numpy as np
import os
from skimage import io
from skimage.transform import resize
from skimage.filters import threshold_otsu

# parameters that you should set before running this script
filter = ['aeroplane', 'bird']       # select class, this default should yield 1489 training and 1470 validation images
voc_root_folder = "/content/VOCdevkit"  # please replace with the location on your laptop where you unpacked the tarball
image_size = 208    # image size that you will use for your network (input images will be resampled to this size), lower if you have troubles on your laptop (hint: use io.imshow to inspect the quality of the resampled images before feeding it into your network!)


# step1 - build list of filtered filenames
annotation_folder = os.path.join(voc_root_folder, "VOC2009/Annotations/")
annotation_files = os.listdir(annotation_folder)
filtered_filenames = []
for a_f in annotation_files:
    tree = etree.parse(os.path.join(annotation_folder, a_f))
    if np.any([tag.text == filt for tag in tree.iterfind(".//name") for filt in filter]):
        filtered_filenames.append(a_f[:-4])

# step2 - build (x,y) for TRAIN/VAL (classification)
classes_folder = os.path.join(voc_root_folder, "VOC2009/ImageSets/Main/")
classes_files = os.listdir(classes_folder)
train_files = [os.path.join(classes_folder, c_f) for filt in filter for c_f in classes_files if filt in c_f and '_train.txt' in c_f]
val_files = [os.path.join(classes_folder, c_f) for filt in filter for c_f in classes_files if filt in c_f and '_val.txt' in c_f]


def build_classification_dataset(list_of_files):
    """ build training or validation set

    :param list_of_files: list of filenames to build trainset with
    :return: tuple with x np.ndarray of shape (n_images, image_size, image_size, 3) and  y np.ndarray of shape (n_images, n_classes)
    """
    temp = []
    train_labels = []
    for f_cf in list_of_files:
        with open(f_cf) as file:
            lines = file.read().splitlines()
            temp.append([line.split()[0] for line in lines if int(line.split()[-1]) == 1])
            label_id = [f_ind for f_ind, filt in enumerate(filter) if filt in f_cf][0]
            train_labels.append(len(temp[-1]) * [label_id])
    train_filter = [item for l in temp for item in l]

    image_folder = os.path.join(voc_root_folder, "VOC2009/JPEGImages/")
    image_filenames = [os.path.join(image_folder, file) for f in train_filter for file in os.listdir(image_folder) if
                       f in file]
    x = np.array([resize(io.imread(img_f,0), (image_size, image_size,3)) for img_f in image_filenames]).astype(
        'float32')
    # changed y to an array of shape (num_examples, num_classes) with 0 if class is not present and 1 if class is present
    y_temp = []
    for tf in train_filter:
        y_temp.append([1 if tf in l else 0 for l in temp])
    y = np.array(y_temp)

    return x, y
def build_seg_dataset(list_of_files):
  
    image_folder = os.path.join(voc_root_folder, "VOC2009/JPEGImages/")
    mask_folder = os.path.join(voc_root_folder, "VOC2009/SegmentationClass/")
    image_filenames = [os.path.join(image_folder, file) for f in list_of_files for file in os.listdir(image_folder) if
                       f in file]
    mask_filenames = [os.path.join(mask_folder, file) for f in list_of_files for file in os.listdir(mask_folder) if
                       f in file]
    x = np.array([resize(io.imread(img_f,0), (image_size, image_size)) for img_f in image_filenames]).astype('float32')
    y = np.array([resize(io.imread(img_f,1), (image_size, image_size)) > 0.01 for img_f in mask_filenames]).astype('float32')
    return x,y
x_train, y_train = build_classification_dataset(train_files)
print('%i training images from %i classes' %(x_train.shape[0], y_train.shape[1]))
x_val, y_val = build_classification_dataset(val_files)
print('%i validation images from %i classes' %(x_val.shape[0],  y_train.shape[1]))
# x_strain, y_strain = build_seg_dataset(open(list_train).read().splitlines())
print('%i training pairs' %(x_strain.shape[0]))
# x_sval, y_sval = build_seg_dataset(open(list_val).read().splitlines())
print('%i validation pairs' %(x_sval.shape[0]))


# from here, you can start building your model open(list_train).read().splitlines()
# you will only need x_train and x_val for the autoencoder
# you should extend the above script for the segmentation task (you will need a slightly different function for building the label images)

463 training images from 2 classes
449 validation images from 2 classes
749 training pairs
750 validation pairs


Saving dataset to drive, only if fresh and regenerated

In [0]:
np.save("/content/drive/My Drive/cvproj/dataset/x_train",x_train)
np.save("/content/drive/My Drive/cvproj/dataset/y_train",y_train)
np.save("/content/drive/My Drive/cvproj/dataset/x_strain",x_strain)
np.save("/content/drive/My Drive/cvproj/dataset/y_strain",y_strain)
np.save("/content/drive/My Drive/cvproj/dataset/x_val",x_val)
np.save("/content/drive/My Drive/cvproj/dataset/y_val",y_val)
np.save("/content/drive/My Drive/cvproj/dataset/x_sval",x_sval)
np.save("/content/drive/My Drive/cvproj/dataset/y_sval",y_sval)

In [0]:
!rsync -a  --no-compress --progress "/content/VOCdevkit" "/content/drive/My Drive/cvproj/dataset/"

Autoencoder 

In [9]:
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
K.clear_session()
image_size=208
input_img = Input(shape=(image_size, image_size, 3)) 

x = Conv2D(32, (3, 3), activation='relu', padding='same',input_shape=(image_size, image_size, 3))(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
encoded1 = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(encoded1)
encoded2 = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded =Conv2D(128,(3, 3), strides=(2,2), padding='same')(encoded2)

x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu',padding='same')(x)
x = Conv2D(32, (3, 3), activation='relu',padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3,3),activation='relu',padding='same')(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
for layer in autoencoder.layers:
   
    layer.trainable = True


autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 208, 208, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 208, 208, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 104, 104, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 104, 104, 32)      9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 104, 104, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 52, 52, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 64)        36928 

In [0]:
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('best_modelautoenc.h5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)
autoencoder.fit( x_train,x_train,batch_size=16,shuffle=True,epochs=100,
                validation_data=(x_val, x_val),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder'),es,mc])

Train on 1489 samples, validate on 1470 samples
Epoch 1/100
1360/1489 [==========================>...] - ETA: 0s - loss: 0.6144

KeyboardInterrupt: ignored

In [0]:
from tensorflow.keras.models import load_model
autoencoder.load_weights("/content/drive/My Drive/cvproj/model_out/model_out/outoencoder150epoch.h5")

Visualize autoenc output

In [0]:
import scipy
from PIL import Image
import imageio
full =np.array(x_val, copy=True)
np.random.shuffle(full)
sel = full[:50]

decoded_imgs = autoencoder.predict(sel)
import matplotlib.pyplot as plt
plt.ioff()
n = 50
for i in range(1,n):
    imageio.imwrite("/content/drive/My Drive/cvproj/outfigautenc/"+ str(i)+"in.png",sel[i])
    imageio.imwrite("/content/drive/My Drive/cvproj/outfigautenc/"+ str(i)+"out.png",decoded_imgs[i])

In [0]:
full =np.array(x_val, copy=True)
np.random.shuffle(full)
sel = full[:10]

decoded_imgs = autoencoder.predict(sel)
import matplotlib.pyplot as plt

n = 10
plt.figure(figsize=(20, 4))
for i in range(1,n):
    # display original
    ax = plt.subplot(2, n, i)
    plt.imshow(sel[i].reshape(208, 208,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(208, 208,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
plt.savefig("/content/drive/My Drive/cvproj/out.jpg")

<Figure size 432x288 with 0 Axes>

In [11]:
%tensorboard --logdir /tmp/autoencoder

UsageError: Line magic function `%tensorboard` not found.


In [0]:
!mkdir -p /model_out
model_json = autoencoder.to_json()
with open("/model_out/outoencoder150.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
autoencoder.save_weights("/model_out/outoencoder150epoch.h5")

In [0]:
!rsync -a -v --no-compress --progress "/model_out" "/content/drive/My Drive/cvproj/model_out/"

sending incremental file list
model_out/
model_out/outoencoder150.json
         10,141 100%    0.00kB/s    0:00:00 (xfr#1, to-chk=1/3)
model_out/outoencoder150epoch.h5
        923,240 100%   58.70MB/s    0:00:00 (xfr#2, to-chk=0/3)

sent 933,803 bytes  received 58 bytes  1,867,722.00 bytes/sec
total size is 933,381  speedup is 1.00


In [0]:
! rm -r /tmp/classif

In [10]:
from tensorflow.keras.layers import Flatten, Dropout
from tensorflow.keras import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import RMSprop

#classif = Sequential()
l = Dropout(rate=0.5)(encoded)
#outl = Conv2D(y_train.shape[1],(13,13),activation='softmax',name="fcimit")(l)
#outl = Reshape((5,))(outl)
l=Flatten()(l)
outl = Dense(5,activation='softmax')(l)
classif= Model(input_img,outl)
for layer in classif.layers[:10]:
  layer.trainable = True


classif.compile(optimizer=optimizers.SGD(), loss='categorical_crossentropy',metrics=['accuracy'])
classif.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 208, 208, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 208, 208, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 104, 104, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 104, 104, 32)      9248      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 104, 104, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 52, 52, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 52, 52, 64)        3692

In [0]:
for layer in classif.layers: 
      if hasattr(layer, 'kernel_initializer'):
          layer.kernel.initializer.run()

AttributeError: ignored

In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=200)
mc = ModelCheckpoint('/content/drive/My Drive/cvproj/best_modelclassifprofull2a.h5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)
classif.fit(x_train, y_train, batch_size=16, epochs=1000, shuffle=True,
                validation_data=(x_val, y_val),
                callbacks=[TensorBoard(log_dir='/content/drive/My Drive/cvproj/classif'),es,mc])


#full =np.array(x_val, copy=True)
#np.random.shuffle(full)
#sel = full[:100]
#classifm=Sequential()
#classifm.add(classif)
#import time
#decoded = classifm.predict_classes(sel,verbose=1)
#import matplotlib.pyplot as plt
#plt.ioff()

#filter = ['aeroplane','bird']
#n = 100
#for i in range(1,n):
#  fig = plt.figure(figsize=(4, 4))
#  plt.imshow(sel[i].reshape(208, 208,3))
#  plt.gray()
#  plt.xlabel(filter[decoded[i]]) 
#  ax=plt.gca()
#  ax.get_yaxis().set_visible(False)
#  ax.get_xaxis().set_ticks([])
#  ax.get_yaxis().set_ticklabels([])
#  plt.savefig("/content/drive/My Drive/cvproj/outfigclassif/" +str(i)+ ".png")
#  plt.close(fig)

Train on 1489 samples, validate on 1470 samples
Epoch 1/1000
1488/1489 [============================>.] - ETA: 0s - loss: 1.6950 - acc: 0.2487
Epoch 00001: val_acc improved from -inf to 0.22925, saving model to /content/drive/My Drive/cvproj/best_modelclassifprofull2a.h5
1489/1489 [==============================] - 157s 106ms/sample - loss: 1.6959 - acc: 0.2485 - val_loss: 1.7400 - val_acc: 0.2293
Epoch 2/1000
1488/1489 [============================>.] - ETA: 0s - loss: 1.7616 - acc: 0.2520

In [0]:
full =np.array(x_val, copy=True)
np.random.shuffle(full)
sel = full[:100]
classifm=Sequential()
classifm.add(classif)
import time
decoded = classifm.predict_classes(sel,verbose=1)
import matplotlib.pyplot as plt
plt.ioff()

filter = ['aeroplane','bird']
n = 100
for i in range(1,n):
  fig = plt.figure(figsize=(4, 4))
  plt.imshow(sel[i].reshape(208, 208,3))
  plt.gray()
  plt.xlabel(filter[decoded[i]]) 
  ax=plt.gca()
  ax.get_yaxis().set_visible(False)
  ax.get_xaxis().set_ticks([])
  ax.get_yaxis().set_ticklabels([])
  plt.savefig("/content/drive/My Drive/cvproj/outfigclassif/" +str(i)+ ".png")
  plt.close(fig)

100/100 [==============================] - 0s 4ms/sample


In [73]:
classif.load_weights("/content/drive/My Drive/cvproj/best_modelclassifpro.h5")
full =np.array(x_val, copy=True)
np.random.shuffle(full)
sel = full[:40]
classifm=Sequential()
classifm.add(classif)
filter = ['aeroplane', 'car', 'chair', 'dog', 'bird']
decoded = classifm.predict_classes(sel,verbose=1)
import matplotlib.pyplot as plt

n = 10
plt.figure(figsize=(20, 4))
for i in range(1,n):
    # display original
    ax = plt.subplot(2, n, i)
    plt.imshow(sel[i].reshape(208, 208,3))
    plt.gray()
    plt.xlabel(filter[decoded[i]]) 
    
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticklabels([])
    
    ax = plt.subplot(2, n, n+i)
    plt.imshow(sel[i+20].reshape(208, 208,3))
    plt.gray()
    plt.xlabel(filter[decoded[i+20]]) 
    ax.get_yaxis().set_visible(False)
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticklabels([])


plt.savefig("/content/drive/My Drive/cvproj/out" + ".jpg")
print(decoded)

40/40 [==============================] - 1s 25ms/sample
[4 3 2 1 2 4 1 1 3 4 1 0 1 1 4 3 3 1 3 3 4 2 0 3 1 1 2 1 1 4 2 1 2 1 0 1 1
 1 0 1]


In [0]:
!mkdir -p /model_out
model_json = classif.to_json()
with open("/model_out/classif400_50drop.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
classif.save_weights("/model_out/classif400_50drop.h5")

In [12]:
from tensorflow.keras.models import clone_model
from tensorflow.keras.layers import *
import tensorflow.keras.backend as K
image_size=208
K.clear_session()
sinput_img = Input(shape=(image_size, image_size, 3)) 
sx = Conv2D(64, (3, 3), activation='relu', padding='same',input_shape=(image_size, image_size, 3))(sinput_img)
sxb = Conv2D(64, (3,3),activation='relu',padding='same')(sx)
sx = MaxPooling2D((2, 2), padding='same')(sxb)
sx1 = Conv2D(128, (3, 3), activation='relu', padding='same')(sx)
sx1 = Conv2D(128, (3, 3), activation='relu', padding='same')(sx1)
sx = MaxPooling2D((2, 2), padding='same')(sx1)
sx2 = Dropout(rate=.5)(sx)
sx2 = Conv2D(256, (3, 3), activation='relu', padding='same')(sx2)
sx2 = Conv2D(256, (3, 3), activation='relu', padding='same')(sx)
sx = MaxPooling2D((2, 2), padding='same')(sx2)
sx = Conv2D(512, (3, 3), activation='relu', padding='same')(sx)
sx = Conv2D(512, (3, 3), activation='relu', padding='same')(sx)
sx = UpSampling2D((2, 2))(sx)
sx = concatenate([sx2,sx], axis = 3)
sx = Conv2D(256, (3, 3), activation='relu', padding='same')(sx)
sx = Conv2D(256, (3, 3), activation='relu', padding='same')(sx)
sx = UpSampling2D((2, 2))(sx)
sx = concatenate([sx1,sx], axis = 3)
sx = Conv2D(128, (3, 3), activation='relu', padding='same')(sx)
sx = Conv2D(128, (3, 3), activation='relu',padding='same')(sx)
sx = UpSampling2D((2, 2))(sx)
sx = concatenate([sxb,sx], axis = 3)
sx = Conv2D(64, (3,3),activation='relu',padding='same')(sx)
sx = Conv2D(64, (3,3),activation='relu',padding='same')(sx)
seg = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(sx)
segautoenc = Model(sinput_img,seg)


segautoenc.compile(optimizers.Adam(lr=0.0001),loss='binary_crossentropy',metrics=['accuracy'])
segautoenc.summary()

W0622 19:02:56.548506 140081001686912 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/nn_impl.py:182: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 208, 208, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 208, 208, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 208, 208, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 104, 104, 64) 0           conv2d_1[0][0]                   
______________________________________________________________________________________________

In [0]:
segautoenc.load_weights("/content/drive/My Drive/cvproj/model_out/Useg-20.hdf5")

In [0]:
import imageio
from skimage import img_as_ubyte


from tensorflow.keras.callbacks import *
class RecordImg(Callback):
    def __init__(self,path="/content/drive/My Drive/cvproj/out",freq=5):
      if path[-1]!='/':
        path += '/'
      self.freq=freq
      self.path=path
    def on_epoch_end(self,epoch, logs={}):
        if epoch %self.freq != 0 :
          return
        model=self.model
        imnr=(2*epoch) % 700
        img = x_sval[imnr:imnr+1]
        imgs = y=y_sval[imnr]
        out = model.predict(img)
        imageio.imwrite(self.path+"epoch"+ str(epoch)+"in.png",img_as_ubyte(img[0]))
        imageio.imwrite(self.path+"epoch"+ str(epoch)+"out.png",img_as_ubyte(out[0]))
        imageio.imwrite(self.path+"epoch"+ str(epoch)+"ground.png",img_as_ubyte(imgs))
    
def getGen(x,y):
    data_gen_args = dict(featurewise_center=True,
                         featurewise_std_normalization=True,
                         rotation_range=20,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         
                         zoom_range=0.2)
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)

    # Provide the same seed and keyword arguments to the fit and flow methods
    seed = 1
    image_datagen.fit(x, augment=True, seed=seed)
    mask_datagen.fit(y[...,np.newaxis], augment=True, seed=seed)

    image_generator = image_datagen.flow(
        x,
        batch_size = 16,
        seed=seed)

    mask_generator = mask_datagen.flow(
        y[...,np.newaxis],
        batch_size = 16,
        seed=seed)
    while True:
        mask = mask_generator.next()
        mask[mask > 0.5] = 1
        mask[mask <= 0.5] = 0
        yield (image_generator.next(),mask)

In [0]:
from tensorflow.keras.callbacks import *


from keras import backend as K



csv = CSVLogger('/content/drive/My Drive/cvproj/seglogd.csv',append=True)
es = EarlyStopping(monitor='val_acc', mode='max', verbose=1, patience=100)
mc = ModelCheckpoint('/content/drive/My Drive/cvproj/best_modelsegUjdl.hdf5', monitor='val_acc', mode='max', save_best_only=True,verbose=1)
mc2 = ModelCheckpoint('/content/drive/My Drive/cvproj/model_out/Usegjdl-{epoch:02d}.hdf5',period=1)
ri= RecordImg(path="/content/drive/My Drive/cvproj/outseg",freq=1)
traingen=getGen(x_strain,y_strain)
valgen=getGen(x_sval,y_sval)
segautoenc.fit_generator(
                traingen,steps_per_epoch=100,epochs=50,
                validation_data=valgen,validation_steps=50,
                callbacks=[TensorBoard(log_dir='/tmp/seg'),es,mc,mc2,ri,csv])

W0618 22:44:56.470298 140121062393728 callbacks.py:875] `period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.


Epoch 1/50
 99/100 [============================>.] - ETA: 0s - loss: 0.3950 - acc: 0.8466
Epoch 00001: val_acc improved from -inf to 0.78927, saving model to /content/drive/My Drive/cvproj/best_modelsegUjdl.hdf5
100/100 [==============================] - 114s 1s/step - loss: 0.3928 - acc: 0.8474 - val_loss: 0.5429 - val_acc: 0.7893


/usr/local/lib/python3.6/dist-packages/skimage/util/dtype.py:135: UserWarning: Possible precision loss when converting from float32 to uint8
  .format(dtypeobj_in, dtypeobj_out))


Epoch 2/50
 99/100 [============================>.] - ETA: 0s - loss: 0.2560 - acc: 0.8954
Epoch 00002: val_acc did not improve from 0.78927
100/100 [==============================] - 81s 807ms/step - loss: 0.2562 - acc: 0.8952 - val_loss: 0.6677 - val_acc: 0.7679
Epoch 3/50
 99/100 [============================>.] - ETA: 0s - loss: 0.2472 - acc: 0.9009
Epoch 00003: val_acc did not improve from 0.78927
100/100 [==============================] - 81s 813ms/step - loss: 0.2463 - acc: 0.9013 - val_loss: 0.6430 - val_acc: 0.7755
Epoch 4/50
 99/100 [============================>.] - ETA: 0s - loss: 0.2323 - acc: 0.9090
Epoch 00004: val_acc did not improve from 0.78927
100/100 [==============================] - 80s 795ms/step - loss: 0.2313 - acc: 0.9094 - val_loss: 0.6757 - val_acc: 0.7821
Epoch 5/50
 99/100 [============================>.] - ETA: 0s - loss: 0.2201 - acc: 0.9143
Epoch 00005: val_acc did not improve from 0.78927
100/100 [==============================] - 78s 784ms/step - loss

In [57]:
index = np.random.randint(100,size=10)
n = 9
plt.figure(figsize=(20, 8))
print(index)
for i in range(1,n):
    
    # display original
    ax = plt.subplot(4, n, i)
    plt.imshow(x_sval[index[i]].reshape(208, 208,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    # display original
    ax = plt.subplot(4, n, i+n)
    plt.imshow(y_sval[index[i]].reshape(208, 208))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(4, n, i + 2*n)
    plt.imshow(res[index[i]].reshape(208, 208))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display reconstruction
    ax = plt.subplot(4, n, i + 3*n)
    plt.imshow(res[index[i]].reshape(208, 208)>threshold_otsu(res[index[i]]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    

plt.savefig("/content/drive/My Drive/cvproj/outseg.png")

In [55]:
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
segautoenc=load_model("/content/drive/My Drive/cvproj/model_out/Useg-20.hdf5")
full =np.array(x_sval, copy=True)
sel = full[0:100]
res=segautoenc.predict(sel)
index = np.random.randint(100,size=10)
n = 10
plt.figure(figsize=(20, 8))
for i in range(1,n):
    
    # display original
    ax = plt.subplot(4, n, i)
    plt.imshow(x_sval[index[i]].reshape(208, 208,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    # display original
    ax = plt.subplot(4, n, i+n)
    plt.imshow(y_sval[index[i]].reshape(208, 208))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(4, n, i + 2*n)
    plt.imshow(res[index[i]].reshape(208, 208))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # display reconstruction
    ax = plt.subplot(4, n, i + 3*n)
    plt.imshow(res[index[i]].reshape(208, 208)>threshold_otsu(res[index[i]]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    
plt.show()
plt.savefig("/content/drive/My Drive/cvproj/outse.png")

KeyboardInterrupt: ignored

In [0]:
list_train = os.path.join(voc_root_folder, "VOC2009/ImageSets/Segmentation/train.txt")
list_val = os.path.join(voc_root_folder, "VOC2009/ImageSets/Segmentation/val.txt")
print(open(list_train).read().splitlines())
len(list_train)
print(train_files)
plt.imshow(y_strain[0])

In [9]:
segautoenc.load_weights("/content/drive/My Drive/cvproj/best_modelseg.h5")

NameError: ignored

In [0]:
model_json = segautoenc.to_json()
with open("/content/drive/My Drive/cvproj/seg300.json", "w") as json_file:
    json_file.write(model_json)

In [0]:
K.clear_session()

In [0]:
print(np.shape(x_sval))

(750, 208, 208, 3)


In [0]:
from tensorflow.keras.models import clone_model
from tensorflow.keras.layers import *
from tensorflow.keras import optimizers
K.clear_session()

ainput_img = Input(shape=(image_size, image_size, 3)) 
sx = Conv2D(32, (3, 3), activation='relu', padding='same',input_shape=(image_size, image_size, 3))(ainput_img)
sx = Conv2D(32, (3,3),activation='relu',padding='same')(sx)
sx = MaxPooling2D((2, 2), padding='same')(sx)
sx = Conv2D(64, (3, 3), activation='relu', padding='same')(sx)
sx = Conv2D(64, (3, 3), activation='relu', padding='same')(sx)
sx = MaxPooling2D((2, 2), padding='same')(sx)
sx = Dropout(rate=.2)(sx)
sx = Conv2D(128, (3, 3), activation='relu', padding='same')(sx)
sx = MaxPooling2D((2, 2), padding='same')(sx)
sx = Conv2D(256, (3, 3), activation='relu', padding='same')(sx)
sx = Conv2D(256, (3, 3), activation='relu', padding='same')(sx)
sx = MaxPooling2D((2, 2), padding='same')(sx)
sx = Conv2D(512, (3, 3), activation='relu', padding='same')(sx)
sx = Conv2D(512, (3, 3), activation='relu', padding='same')(sx)
sx = UpSampling2D((2, 2))(sx)
sx = Conv2D(256, (3, 3), activation='relu', padding='same')(sx)
sx = Conv2D(256, (3, 3), activation='relu', padding='same')(sx)
sx = UpSampling2D((2, 2))(sx)
sx = Conv2D(128, (3, 3), activation='relu', padding='same')(sx)
sx = Conv2D(128, (3, 3), activation='relu', padding='same')(sx)
sx = UpSampling2D((2, 2))(sx)
#sx = concatenate([sx1,sx], axis = 3)
sx = Conv2D(64, (3, 3), activation='relu', padding='same')(sx)
sx = Conv2D(64, (3, 3), activation='relu',padding='same')(sx)
sx = UpSampling2D((2, 2))(sx)
#sx = concatenate([sxb,sx], axis = 3)
sx = Conv2D(32, (3,3),activation='relu',padding='same')(sx)
sx = Conv2D(32, (3,3),activation='relu',padding='same')(sx)
decoded = Conv2D(3, (3,3), activation='sigmoid', padding='same')(sx)
autoencpro = Model(ainput_img,decoded)


autoencpro.compile(optimizer=optimizers.SGD(lr=0.1),loss='mse',metrics=['accuracy'])
autoencpro.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 208, 208, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 208, 208, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 208, 208, 32)      9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 104, 104, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 104, 104, 64)      18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 104, 104, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 52, 52, 64)        0     

In [0]:
def getGen(x):
    data_gen_args = dict(featurewise_center=True,
                         featurewise_std_normalization=True,
                         rotation_range=20,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         
                         zoom_range=0.2)
    image_datagen = ImageDataGenerator(**data_gen_args)
    image_datagen.fit(x)
    generator = image_datagen.flow(
    x,x,batch_size=16)
    return generator
  
from tensorflow.keras.callbacks import TensorBoard

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('best_modelautoenc.hdf5', monitor='val_loss', mode='min', save_best_only=True,verbose=1)

autoencpro.fit(x_train,x_train,batch_size=64,epochs=100,shuffle=True,
                         validation_data=(x_val, x_val),
                         callbacks=[TensorBoard(log_dir='/tmp/autoencoder'),es,mc])

Train on 1489 samples, validate on 1470 samples
Epoch 1/100
1472/1489 [============================>.] - ETA: 0s - loss: 0.0731 - acc: 0.4156
Epoch 00001: val_loss improved from inf to 0.07220, saving model to best_modelautoenc.hdf5
1489/1489 [==============================] - 27s 18ms/sample - loss: 0.0730 - acc: 0.4161 - val_loss: 0.0722 - val_acc: 0.4996
Epoch 2/100
1472/1489 [============================>.] - ETA: 0s - loss: 0.0719 - acc: 0.4984
Epoch 00002: val_loss improved from 0.07220 to 0.07118, saving model to best_modelautoenc.hdf5
1489/1489 [==============================] - 28s 19ms/sample - loss: 0.0718 - acc: 0.4977 - val_loss: 0.0712 - val_acc: 0.5004
Epoch 3/100
1472/1489 [============================>.] - ETA: 0s - loss: 0.0712 - acc: 0.4986
Epoch 00003: val_loss improved from 0.07118 to 0.07056, saving model to best_modelautoenc.hdf5
1489/1489 [==============================] - 27s 18ms/sample - loss: 0.0711 - acc: 0.4979 - val_loss: 0.0706 - val_acc: 0.5004
Epoch 4/

In [0]:
full =np.array(x_train, copy=True)
np.random.shuffle(full)
sel = full[:10]

decoded_imgs = autoencpro.predict(sel)
import matplotlib.pyplot as plt

n = 10
plt.figure(figsize=(20, 4))
for i in range(1,n):
    # display original
    ax = plt.subplot(2, n, i)
    plt.imshow(sel[i].reshape(208, 208,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + n)
    plt.imshow(decoded_imgs[i].reshape(208, 208,3))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
plt.savefig("/content/drive/My Drive/cvproj/out.jpg")

<Figure size 432x288 with 0 Axes>

In [58]:
segautoenc.summary

<bound method Network.summary of <tensorflow.python.keras.engine.training.Model object at 0x7f66c9f0ee48>>